In [1]:
import pandas as pd
from yahoo_fin.stock_info import get_data
import datetime

In [2]:
START = TRAIN_START = datetime.datetime(2018,1,1)
TEST_START = datetime.datetime(2020,1,5)
END = datetime.datetime(2020,12,31)

### Download cryptocurrency(CC) market 100 index from [Bitwise](https://www.bitwiseinvestments.com/indexes/Bitwise-100) which covers 93% of the CC market.

In [3]:
CC100_Index = pd.read_csv("indexReturns-BITW100.csv")
CC100_Index['Date'] = pd.to_datetime(CC100_Index['Date']).dt.tz_localize(None)
CC100_Index = CC100_Index.sort_values('Date').set_index('Date')
CC100_Index = CC100_Index.truncate(before = START,after = END)
CC100_Index = CC100_Index.resample('W').last()[:-1]

In [4]:
CC100_Index

,BITW100
Date,
2018-01-07,34027.35
2018-01-14,30534.28
2018-01-21,25761.58
2018-01-28,22724.66
2018-02-04,18134.05
...,...
2020-11-29,18386.98
2020-12-06,19257.57
2020-12-13,19002.90


In [5]:
Test_TimeList = CC100_Index.loc[TEST_START:].index

#### VET Deleted due to late ICO (in Aug 2018)

In [6]:
CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
CC_temp_list = CC_string.split(",")
CC_list = [i+'-USD' for i in CC_temp_list]

In [ ]:
data_all = pd.DataFrame()
for i in CC_list:
    print(i)
    data = get_data(i,start_date = START,end_date = END).resample('W').last()[:-1]
    data_all = pd.concat([data_all,data])
data_all.index.names = ['date']
data_all

ADA-USD
ADX-USD
AE-USD
ANT-USD
ARDR-USD
ARK-USD
BAT-USD
BCH-USD
BCN-USD
BNB-USD
BNT-USD
BTC-USD
BTG-USD
BTS-USD
CVC-USD


In [ ]:
date = Test_TimeList[0]
top10_liquid = data_all.loc[date].sort_values('volume',ascending= False).iloc[:10]
top10_liquid

In [ ]:
data_test = data_all.copy()
data_test = data_test.reset_index().set_index(['ticker','date'])
data_test

# 先搞清楚smf.ols的参数设置，已经数据是否允许nan（算法）

In [ ]:
regression = pd.DataFrame(columns=['beta', 'rse'])
for ticker in top10_liquid['ticker']:
    df = data_test['']
    CAPM_model = smf.ols(formula = ticker+' ~ MarketReturns', data = result)
    CAPM_fit = CAPM_model.fit()
    beta = CAPM_fit.params['MarketReturns']
    rse = CAPM_fit.bse['MarketReturns']
    dict = {'beta': beta, 'rse': rse} 
    df = pd.DataFrame(dict, index = ticker)
    regression = regression.append(df)
    
print(regression)